In [ ]:
# Make sure you have transformers and torch installed
# pip install transformers torch

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Specify the model ID and load the model and tokenizer
# Using a smaller, faster model for this example.
model_id = "LiquidAI/LFM2-350M"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# --- Zero-Shot Prompt ---
# We ask the model to classify the sentiment of a sentence
# without providing any examples.
review_text = "This movie was absolutely fantastic! The acting was superb."
prompt = f"""
Classify the sentiment of the following movie review.
Options: Positive, Negative, Neutral

Review: "{review_text}"
Sentiment:
"""

# Prepare the input for the model
messages = [{"role": "user", "content": prompt}]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

# Generate the response
# We use max_new_tokens to limit the output to a single word.
outputs = model.generate(
    input_ids,
    max_new_tokens=5,
    do_sample=True,
    temperature=0.1,
)

# Decode and print the result
response = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
print(f"Movie Review: {review_text}")
print(f"Model's Predicted Sentiment: {response.strip()}")

# Expected output will likely be 'Positive'


: 